## Develop rules

In [2]:
# import packages that we will need
from nlp_pneumonia_utils import read_doc_annotations
from DocumentClassifier import DocumentClassifier
from nlp_pneumonia_utils import list_errors
from visual import Vis
from visual import snippets_markup
from visual import view_pycontext_output
from visual import display_doc_text
# packages for interaction
from IPython.display import display, HTML
import ipywidgets

### training set

In [19]:
pos_doc_type='SOCIAL_DOC'
annotated_doc_map = read_doc_annotations(archive_file='data/train.zip', pos_type=pos_doc_type)
print('Total Annotated Documents : {0}'.format(len(annotated_doc_map)))

Reading annotations from file : data/train.zip
Opening local file : data/train.zip
Total Annotated Documents : 1


### Rules & initiation 

In [21]:
# Test rules

#target_rules='KB/social_kb.yml'
#context_rules='KB/general_modifiers.yml'
#feature_inference_rule='KB/featurer_inferences.csv'
#document_inference_rule='KB/doc_inferences.csv'

target_rules='''
Comments: ''
Direction: ''
Lex: isolate
Regex: ''
Type: ISOLATION
---
Comments: ''
Direction: ''
Lex: isolated
Regex: ''
Type: ISOLATION
---
Comments: ''
Direction: ''
Lex: isolating
Regex: ''
Type: ISOLATION
---
Comments: ''
Direction: ''
Lex: isolation
Regex: ''
Type: ISOLATION
---
Comments: ''
Direction: ''
Lex: lives alone
Regex: ''
Type: ISOLATION
'''
#
context_rules='''
Comments: ''
Direction: forward
Lex: 'no'
Regex: ''
Type: DEFINITE_NEGATED_EXISTENCE
---
Comments: ''
Direction: forward
Lex: 'low suspicion'
Regex: ''
Type: DEFINITE_NEGATED_EXISTENCE
---
Comments: ''
Direction: forward
Lex: 'denies'
Regex: ''
Type: DEFINITE_NEGATED_EXISTENCE
---
Comments: ''
Direction: forward
Lex: 'r/out'
Regex: ''
Type: DEFINITE_NEGATED_EXISTENCE
---
Comments: ''
Direction: forward
Lex: 'ruled out'
Regex: ''
Type: DEFINITE_NEGATED_EXISTENCE
'''
#
feature_inference_rule='''
#Conclusion type, Evidence type, Modifier values associated with the evidence
NEGATED_CONCEPT,ISOLATION,DEFINITE_NEGATED_EXISTENCE
'''
#
document_inference_rule='''
#Conclusion Type at document level, Evidence type at mention level
ISOLATION_DOC,ISOLATION
#Default document type
NO_ISOLATION
'''

In [24]:
classifier = DocumentClassifier(targets=target_rules, modifiers=context_rules,
                               feature_inference_rule=feature_inference_rule, 
                               document_inference_rule=document_inference_rule,
                               expected_values=[pos_doc_type]) 
classifier.reset_saved_predictions()

### clasification

In [25]:
print('****************')
print('Performance for Classifier :')
current_false_negatives, current_false_positives, measurements,confusion_matrix_df = classifier.eval(annotated_doc_map)
print(measurements)
display(confusion_matrix_df)
print('****************')

****************
Performance for Classifier :
Start to evaluate against reference standards...


AttributeError: 'str' object has no attribute 'getLiteral'

### False Negative Viewer - standards

In [ ]:
fn_docs=dict((k, v) for k, v in annotated_doc_map.items() if k in current_false_negatives)
display(HTML(snippets_markup(fn_docs,'SOCIAL_DOC')))

### False Negative Viewer - annotations

In [ ]:
# set up the visualizer for pyConText output
vis=Vis(MODIFIERS_FILE_PATH)
fn_docs = dict((k,v) for k, v in classifier.saved_markups_map.items() if k in current_false_negatives)
view_pycontext_output(fn_docs,vis)

### False positive Viewer - standards

In [ ]:
fp_docs=dict((k, v) for k, v in annotated_doc_map.items() if k in current_false_positives)
display(HTML(snippets_markup(fp_docs,'SOCIAL_DOC')))

### False positive Viewer - annotations

In [ ]:
fp_docs = dict((k,v) for k, v in classifier.saved_markups_map.items() if k in current_false_positives)
view_pycontext_output(fp_docs,vis)

### test a sentence after revise the rule

In [ ]:
# Refresh the classifier with updated rules
classifier = DocumentClassifier(TARGETS_FILE_PATH, MODIFIERS_FILE_PATH,
                               FEATURE_INFERENCER_FILE_PATH, DOC_INFERENCER_FILE_PATH,
                               {pos_doc_type})

str='''sentences will be tested here'''
res=classifier.predict(str)
print("Positive" if res==1 else "Negative")
view_pycontext_output(classifier.get_last_doc_markups(), vis)


### Test Set

In [ ]:
test_doc_map = read_doc_annotations(archive_file='data/test.zip', pos_type=pos_doc_type)
print('****************')
print('Performance for Classifier on test set:')
current_false_negatives, current_false_positives, measurements,confusion_matrix_df = classifier.eval(annotated_doc_map)
print(measurements)
display(confusion_matrix_df)
print('****************')